In [1]:
import tensorflow as tf
import keras
import numpy as np
import os
from PIL import Image

model = keras.applications.MobileNet(
    input_shape=(224, 224, 3),
    alpha=0.25,
    include_top=True,
    weights="imagenet",
    classifier_activation='softmax'
)

def preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img_resized = img.resize((224, 224))
    img_np = np.array(img_resized).astype(np.float32)
    img_np = np.expand_dims(img_np, axis=0)
    return (img_np/127.5 - 1.0)

def get_image_paths_from_folder(folder_path, num_images=100):
    image_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".JPEG"):
                image_paths.append(os.path.join(root, file))
            if len(image_paths) >= num_images:
                return image_paths
    return image_paths

folder = "./image_data/train"  # Calibration data, you can download from https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz
calibration_images = get_image_paths_from_folder(folder, num_images=300)

def representative_data_gen():
    for image_path in calibration_images:
        img_np = preprocess_image(image_path)
        yield [img_np]


converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

converter.representative_dataset = representative_data_gen

tflite_model = converter.convert()

tflite_model_path = "mobilenet_v1.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

2108140/2108140 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Saved artifact at '/tmp/tmpj8q9bv13'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)
Captures:
  139341962154768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139341945716560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139341945715600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139341945716368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139341945716176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139341945714256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139341943473168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139341943472400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139341943473552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139341943473744: TensorSpec(shap

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import requests

model_url = "https://storage.googleapis.com/tensorflow/keras-applications/mobilenet/mobilenet_2_5_224_tf.h5"
drive_path = "/content/drive/MyDrive/mobilenet_2_5_224_tf.h5" # Changed this line to download to the root directory of MyDrive

response = requests.get(model_url)
response.raise_for_status()  # Raise an exception for bad status codes

with open(drive_path, 'wb') as f:
    f.write(response.content)

print(f"Downloaded model to {drive_path}")

Downloaded model to /content/drive/MyDrive/mobilenet_2_5_224_tf.h5
